# LeetCode 108. Convert Sorted Array to Binary Search Tree (Easy)

> see: https://leetcode.com/problems/convert-sorted-array-to-binary-search-tree/

Given an array where elements are sorted in ascending order, convert it to a height balanced BST.

For this problem, a height-balanced binary tree is defined as a binary tree in which the depth of the two subtrees of every node never differ by more than 1.

Example:

```
Given the sorted array: [-10,-3,0,5,9],

One possible answer is: [0,-3,9,-10,null,5], which represents the following height balanced BST:

      0
     / \
   -3   9
   /   /
 -10  5

```

## 1. CCJ's Solution

> Basically, the height-balanced restriction means that at each step one has to pick up the number in the middle as a root.

> It's known that inorder traversal of BST is an array sorted in the ascending order.

In [1]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution(object):
    def sortedArrayToBST(self, nums):
        """
        :type nums: List[int]
        :rtype: TreeNode
        """
        
        if nums is None or len(nums) == 0:
            return None
        elif len(nums) == 1:
            return TreeNode(nums[0])
        else:
            n = len(nums)
            k = n//2
            val = nums[k]
            #print (n, k, "root = ", val)
            root = TreeNode(val)
            ## preorder traversal: node -> left -> right
            # pay attention to the index included or excluded: 
            root.left =  self.sortedArrayToBST(nums[0:k])
            root.right = self.sortedArrayToBST(nums[k+1:n])
            return root

### 1.1 Results

<img src="../files/leet-code-108-2020-11-07.png" alt="drawing" width="500"/>


### 1.2 Errors You Possibly Make

Pay attention to the index included or excluded: with index k as current root position, 

- the first half will be $0, 1, \dots, k-1$, thus in Python we use nums[0:k] instead of [0:k-1].
- the last half will be $k+1:$ in Python.

## 2. Solutions and Analysis from `LeetCode Solution`

## 2.1 How to Traverse the Tree. DFS: Preorder, Inorder, Postorder; BFS.

There are two general strategies to traverse a tree:

- Depth First Search (DFS): 

 - In this strategy, we adopt the depth as the priority, so that one would start from a root and reach all the way down to certain leaf, and then back to root to reach another branch.

 - The DFS strategy can further be distinguished as preorder, inorder, and postorder depending on the relative order among the root node, left node and right node.

- Breadth First Search (BFS):
 - We scan through the tree level by level, following the order of height, from top to bottom. The nodes on higher level would be visited before the ones with lower levels.


On the following figure the nodes are enumerated in the order you visit them, please follow 1-2-3-4-5 to compare different strategies.


<img src="../files/bfs_dfs.png" alt="drawing" width="800"/>

## 2.2 Construct BST from Inorder Traversal: Why the Solution is Not Unique

> It's known that inorder traversal of BST is an array sorted in the ascending order.

Having sorted array as an input, we could rewrite the problem as `Construct Binary Search Tree from Inorder Traversal`.

> Does this problem have a unique solution, i.e. could inorder traversal be used as a unique identifier to encore/decode BST? The answer is no.

Here is the funny thing about BST. Inorder traversal is not a unique identifier of BST. At the same time both preorder and postorder traversals are unique identifiers of BST. From these traversals one could restore the inorder one: inorder = sorted(postorder) = sorted(preorder), and inorder + postorder or inorder + preorder are both unique identifiers of whatever binary tree.

So, the problem "sorted array -> BST" has multiple solutions.

<img src="../files/bst2.png" alt="drawing" width="800"/>

Here we have an additional condition: the tree should be height-balanced, i.e. the depths of the two subtrees of every node never differ by more than 1.

> Does it make the solution to be unique? Still no.

<img src="../files/height.png" alt="drawing" width="800"/>

Basically, the height-balanced restriction means that at each step one has to `pick up the number in the middle as a root`. That works fine with arrays containing odd number of elements but there is no predefined choice for arrays with `even` number of elements.

<img src="../files/pick.png" alt="drawing" width="800"/>

One could choose left middle element, or right middle one, and both choices will lead to different height-balanced BSTs. 

Let's see that in practice: in `Approach 1` we will always pick up left middle element, in Approach 2 - right middle one. That will generate different BSTs but both solutions will be accepted.

## 2.3 Approach 1: Preorder Traversal: Always Choose Left Middle Node as a Root

**Algorithm**

<img src="../files/bst_pick_left.png" alt="bst_pick_left" width="600"/>


- Implement helper function `helper(left, right)`, which constructs BST from `nums` elements between indexes `left` and `right`:

- If left > right, then there is no elements available for that subtree. Return None.

 - Pick left middle element: p = (left + right) // 2.

 - Initiate the root: root = TreeNode(nums[p]).
 
 - Compute recursively left and right subtrees: root.left = helper(left, p - 1), root.right = helper(p + 1, right).

- Return helper(0, len(nums) - 1).



In [6]:
# Definition for a binary tree node.
class TreeNode(object):
     def __init__(self, val=0, left=None, right=None):
         self.val = val
         self.left = left
         self.right = right

class Solution:
    def sortedArrayToBST(self, nums):
        """
        :type nums: List[int]
        :rtype: TreeNode
        """
        def helper(left, right):
            if left > right:
                return None

            # always choose left middle node as a root
            p = (left + right) // 2

            # preorder traversal: node -> left -> right
            root = TreeNode(nums[p])
            root.left = helper(left, p - 1)
            root.right = helper(p + 1, right)
            return root
        
        return helper(0, len(nums) - 1)

**Complexity Analysis**

- Time complexity: $\mathcal{O}(N)$ since we visit each node exactly once.

- Space complexity: $\mathcal{O}(N)$. Here $\mathcal{O}(N)$ to keep the output, and $\mathcal{O}(\log N)$ for the recursion stack.

<img src="../files/leet-code-108-2020-11-07-try2.png" alt="drawing" width="500"/>

## 2.4 Approach 2: Preorder Traversal: Always Choose Right Middle Node as a Root

**Algorithm**

<img src="../files/bst_pick_right.png" alt="drawing" width="600"/>


- Implement helper function `helper(left, right)`, which constructs BST from nums elements between indexes `left` and `right`:

 - If `left` > `right`, then there is no elements available for that subtree. Return None.

 - Pick right middle element: 
  - p = (left + right + 1) // 2.

 - Initiate the root: root = TreeNode(nums[p]).

 - Compute recursively left and right subtrees: root.left = helper(left, p - 1), root.right = helper(p + 1, right).

- Return helper(0, len(nums) - 1).

In [2]:
class Solution:
    def sortedArrayToBST(self, nums):        
        def helper(left, right):
            if left > right:
                return None
            
            # always choose right middle node as a root
            p = (left + right + 1) // 2
            
            # preorder traversal: node -> left -> right
            root = TreeNode(nums[p])
            root.left = helper(left, p - 1)
            root.right = helper(p + 1, right)
            return root
        
        return helper(0, len(nums) - 1)

Complexity is the same as before.